# Python Średniozaawansowany

#### Instrukcja instalacji VS Code <a class="anchor" id="install"></a>

 - Zainstaluj narzędzia [Python](https://www.python.org/downloads/).

 - Pobierz [VS Code](https://code.visualstudio.com/).

 - Zainstaluj rozszerzenie [Python](https://marketplace.visualstudio.com/items?itemName=ms-python.python).

 - Zainstaluj rozszerzenie [Jupyter](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter).

  - Zainstaluj rozszerzenie [Git Extension Pack](https://marketplace.visualstudio.com/items?itemName=donjayamanne.git-extension-pack).

#### Potrzebne importy

 - [matplotlib](https://matplotlib.org/) - wymagana instalacja
    * [pyplot](https://matplotlib.org/stable/tutorials/introductory/pyplot.html)
 - [numpy](https://numpy.org/) - wymagana instalacja
 - os
 - [inspect](https://docs.python.org/3/library/inspect.html) - biblioteka zawierająca [funkcjonalność przydatną przy analizowaniu struktur klas](https://docs.python.org/3/library/inspect.html#classes-and-functions).
 - [abc](https://docs.python.org/3/library/abc.html) - biblioteka do abstrakcji klasowej


In [1]:
from abc import ABC, abstractmethod
import traceback
import os
from inspect import getmro

In [2]:
from matplotlib import pyplot as plt
import numpy as np

## Wyjątki

### [Wyjątki](https://en.wikipedia.org/wiki/Exception_handling) są zdarzeniami, które pojawiają się podczas wykonania programu i przerywają jego normalny przebieg. Technicznie są to obiekty reprezentujące dany błąd.

In [3]:
a = 5
b = 0
print(a/b)

ZeroDivisionError: division by zero

##### Wszystkie wbudowane, nie opuszczające systemu [wyjątki](https://docs.python.org/3/library/exceptions.html) dziedziczą po klasie `Exception`. Wyjątki tworzone przez programistę również powinny dziedziczyć po tej klasie.

##### Najczęściej spotykane wyjątki to

 - `AssertionError` - niespełnienie warunku operacji [assert](https://docs.python.org/3/reference/simple_stmts.html#the-assert-statement)
 - `AttributeError` - odwołanie do nieistniejącego atrybutu
 - `IOError` - błąd operacji wejścia/wyjścia
 - `IndexError` - odwołanie do kolekcji za pomocą indeksu poza zakresem
 - `ImportError` - problem z załadowaniem modułu
 - `KeyError` - odwołanie do słownika nieistniejącym kluczem
 - `ValueError` - przekazanie do funkcji argumentu o niepoprawnej wartości
 - `ZeroDivisionError` - dzielenie przez zero

### Obsługa wyjątków

##### Wyjątki obsługujemy za pomocą słów kluczowych:
 - `try` - poprzedza obszar sprawdzany pod kątem wyrzucania wyjątków,
 - `except` - poprzedza obszar obsługi wyłapanego wyjątku,
 - `finally` - poprzedza obszar, który jest wykonywany pod koniec obsługi, niezależnie od tego, czy wyjątek został wyrzucony czy też nie
 - `raise` - wyrzuca zadany wyjątek

In [4]:
for i in [0.5, 0.3, 0]:
    result = 0
    try:
        if i == 0:
            result = 10000000
        else:
            result = 10/i
    except:
        result = 10000000
    print(i, " ", result)

0.5   20.0
0.3   33.333333333333336
0   10000000


In [5]:
def root4(x):
    return x ** 0.25

x = "Ala ma kota"

try:
    print(root4(x))
except:
    print("Hej nie umiem tego potęgować")

Hej nie umiem tego potęgować


In [6]:
for i in [0.5, 0.3, 0.001]:
    result = 0
    try:
        result = 10/(max(0.1,i) - 0.1)
    except:
        result = 10000000
        raise ValueError("Value lower than 0.1")
    finally:
        print(i, " ", result)


0.5   25.0
0.3   50.00000000000001
0.001   10000000


ValueError: Value lower than 0.1

##### Do każdego bloku `try` możemy przypisać wiele bloków `except`.

In [7]:
l = [5, 0, 6, 2]

for i in range(5):
    try:
        a = 1/l[i]
        print(a)
        assert a >= 0.2
    except KeyError:
        print("Wrong key")
    except (ZeroDivisionError):
        print("Oopsie, whoopsie. You tried to divide by zero")
    except (IndexError, KeyError):
        print("I went too far")
    except:
        print("What did just happened here?")

0.2
Oopsie, whoopsie. You tried to divide by zero
0.16666666666666666
What did just happened here?
0.5
I went too far


##### Klauzulę `finally` zazwyczaj używamy, gdy po wykonaniu zadania musimy zwolnić pewne zasoby, niezależnie, czy procedura powiodła się czy nie. Na przykład, obsługa pliku.

In [8]:
dummy_file_name = "dummy.file"

try:
    file = open(dummy_file_name, "r")
    print(file.read())
except:
    print("File is not readable")
finally:
    file.close()
    if os.path.exists(dummy_file_name):
        os.remove(dummy_file_name)


File is not readable


NameError: name 'file' is not defined

### Manager kontekstu

##### Zamiast tego możemy użyć wyrażenia [with](https://docs.python.org/3/reference/compound_stmts.html#the-with-statement) zwanego managerem kontekstu, które obsługuje obiekty posiadające metody `__enter__` i `__exit__`

In [9]:
dummy_file_name = "dummy.file"

with open(dummy_file_name, "w") as file:
    file.write("Alamakotaasierotkamarysia")

if os.path.exists(dummy_file_name):
    os.remove(dummy_file_name)

with open(dummy_file_name, "r") as file:
    print(file.read())

FileNotFoundError: [Errno 2] No such file or directory: 'dummy.file'

In [10]:
dummy_file_name = "dummy.file"

class AutodestructingFile:

    def __init__(self, address) -> None:
        self.address = address
        self.writer = open(address, "w")
        self.reader = open(address, "r")

    def write(self, text):
        self.writer.write(text)

    def read(self):
        return self.reader.read()

    def close(self):
        self.reader.close()
        self.writer.close()
        if os.path.exists(self.address):
            os.remove(self.address)

    def __enter__(self):
        return self

    def __exit__(self, type, value, traceback):
        print(traceback)
        self.close()


with AutodestructingFile(dummy_file_name) as file:
    file.write("lol")
    print(file.read())

    raise AssertionError("lol")
    print("message ignored")

AssertionError: lol

In [11]:
from contextlib import contextmanager

dummy_file_name = "dummy.file"

class ReadWriteFile:
    def __init__(self, address) -> None:
        self.address = address
        self.writer = open(address, "w")
        self.reader = open(address, "r")

    def write(self, text):
        self.writer.write(text)

    def read(self):
        return self.reader.read()

@contextmanager
def autodestructingFile(address):
    file = ReadWriteFile(address)
    try:
        yield file
    finally:
        file.reader.close()
        file.writer.close()
        if os.path.exists(address):
            os.remove(address)


with autodestructingFile(dummy_file_name) as file:
    file.write("lol")
    print(file.read())

    raise Exception("lol")
    print("message ignored")

Exception: lol

### [Rzucanie wyjątków](https://docs.python.org/3/tutorial/errors.html#raising-exceptions)

##### W celu rzucenia wyjątku wykorzystujemy wyrażenie `raise`.

In [12]:
raise Exception("My random exception")

Exception: My random exception

### Tworzenie własnych wyjątków

In [13]:
class CustomException1(Exception):
    pass

try:
    raise CustomException1("Some custom exception")
except:
    traceback.print_exc()

class CustomException2(Exception):
    def __init__(self) -> None:
        super().__init__("My hardcoded message")

    def sing(self):
        print("In the marry month of June")

try:
    raise CustomException2()
except CustomException2 as e:
    traceback.print_exc()
    e.sing()

In the marry month of June


Traceback (most recent call last):
  File "C:\Users\micha\AppData\Local\Temp/ipykernel_4936/2871786223.py", line 5, in <module>
    raise CustomException1("Some custom exception")
CustomException1: Some custom exception
Traceback (most recent call last):
  File "C:\Users\micha\AppData\Local\Temp/ipykernel_4936/2871786223.py", line 17, in <module>
    raise CustomException2()
CustomException2: My hardcoded message


In [14]:
def fact(n):
    if n == 0:
        return 1
    else:
        return fact(n - 1) * n

## [Programowanie obiektowe](https://en.wikipedia.org/wiki/Object-oriented_programming)

##### Podstawowe koncepcje to

 - [dziedziczenie](https://en.wikipedia.org/wiki/Inheritance_(object-oriented_programming)),
 - [kompozycja](https://en.wikipedia.org/wiki/Object_composition),
 - [abstrakcja](https://en.wikipedia.org/wiki/Abstraction_(computer_science)),
 - [hermetyzacja](https://en.wikipedia.org/wiki/Encapsulation_(computer_programming)),
 - [polimorfizm](https://en.wikipedia.org/wiki/Polymorphism_(computer_science)).

### Dziedziczenie

Jeżeli pewna klasa posiada funkcjonalność, która jest nam potrzebna, ale chcielibyśmy jeszcze dodać coś od siebie, możemy rozszerzyć klasę tworząc nową, która dziedziczy funkcjonalności klasy bazowej.

In [15]:
class Animal:
    def __init__(self, name) -> None:
        self.name = name

    def eat(self, food):
        print(self.name, ":Nom, nom, nom. Tasty", food)

class Bird(Animal):
    def fly(self):
        print(self.name, ": I can fly")

animal = Animal("Creature")
animal.eat("other animal")

bird = Bird("Tweety")
bird.eat("grain")
bird.fly()

def feed(animal: Bird, food: object) -> None:
    animal.eat(food)
    animal.fly()

feed(bird, "grain")
bird.__str__()

Creature :Nom, nom, nom. Tasty other animal
Tweety :Nom, nom, nom. Tasty grain
Tweety : I can fly
Tweety :Nom, nom, nom. Tasty grain
Tweety : I can fly


'<__main__.Bird object at 0x00000238D9F01640>'

Możemy sprawdzić pochodzenie naszej klasy za pomocą funkcji `getmro` z pakietu `inspect`.

In [16]:
print(getmro(type(bird)))
print(getmro(type(getmro)))
print(getmro(AttributeError))
print(getmro(type(5.0)))

(<class '__main__.Bird'>, <class '__main__.Animal'>, <class 'object'>)
(<class 'function'>, <class 'object'>)
(<class 'AttributeError'>, <class 'Exception'>, <class 'BaseException'>, <class 'object'>)
(<class 'float'>, <class 'object'>)


Możemy też dziedziczyć po wielu klasach przejmując atrybuty każdej z nich.

In [17]:
class Animal:
    def __init__(self, name) -> None:
        self.name = name
        print("I am an animal")

    def eat(self, food):
        print(self.name, ": Nom, nom, nom. Tasty", food)

class Oviparous(Animal):
    def __init__(self, name) -> None:
        super().__init__(name)
        print("I am an oviparous")

    def bearEgg(self):
        print(self.name, ": plop")

    def breath(self):
        print(self.name, ": wheez")

class Mammal(Animal):
    def __init__(self, name) -> None:
        super().__init__(name)
        print("I am a mammal")

    def giveMilk(self):
        print(self.name, ": blurp")

    def breath(self):
        print(self.name, ": phew")

class Platypus(Mammal, Oviparous):
    def __init__(self, name) -> None:
        super().__init__(name)
        print("I am a platypus")

    def sting(self):
        print(self.name, ": sting")

perry = Platypus("Perry")

perry.bearEgg()
perry.giveMilk()
perry.eat("algae")
perry.breath()

print(getmro(Platypus))

I am an animal
I am an oviparous
I am a mammal
I am a platypus
Perry : plop
Perry : blurp
Perry : Nom, nom, nom. Tasty algae
Perry : phew
(<class '__main__.Platypus'>, <class '__main__.Mammal'>, <class '__main__.Oviparous'>, <class '__main__.Animal'>, <class 'object'>)


**TODO** Zauważmy, że powyższa klasa rozwiązuje dziedziczenie wpierw rozpatrując klasę `Base1`. Oznacza to, że jeśli klasy `Base1` i `Base2` będą miały takie samo pole, `Child` przejmie tylko pole od `Base1`.

### Kompozycja

Zamiast dziedziczyć wszystkie atrybuty możemy wkomponować jedną klasę w drugą tworząc obiekt tej pierwszej jako pole drugiej. Następnie pole to możemy wykorzystać do implementacja własnych metod.

In [18]:
from __future__ import annotations

class Hero():
    def __init__(self, name: str) -> None:
        self.name = name

    def introduce(self) -> str:
        return f"I am {self.name}"

class Mage(Hero):
    def __init__(self, name: str, staff: Staff) -> None:
        super().__init__(name)
        self.staff = staff

    def equipStaff(self, staff: Staff):
        self.staff = staff

    def castSpell(self) -> str:
        if self.staff:
            return self.staff.use()
        else:
            return None

class Staff:
    def use(self):
        pass

    def setOwner(self, mage: Mage) -> None:
        self.mage = mage

class FireWand(Staff):
    def use(self) -> str:
        return "Fireball!"

class FrostWand(Staff):
    def use(self) -> str:
        return "Brrr"

mage1 = Mage("Master", FireWand())
mage2 = Mage("Timothy", FrostWand())

print(mage1.introduce(), mage1.castSpell())
print(mage2.introduce(), mage2.castSpell())

I am Master Fireball!
I am Timothy Brrr


### Abstrakcja

Czasami chcemy stworzyć wzór klasy bez implementowania poszczególnych jej elementów. Przkładowo, chcę stworzyć klasę `Collection`, która pozwoli użytkownikowi na dostęp do elementów, ale pozwoli również zaimplementować różne kontenery przechowujące elementy.

Nie jest to podstawowy koncept w Pythonie ze względu na brak [silnej typizacji](https://en.wikipedia.org/wiki/Strong_and_weak_typing), stąd musimy wykorzystać do tego pakiet `abc`.

In [19]:
class Collection:
    def __init__(self, elements) -> None:
        pass

    def getNext(self):
        pass

c = Collection([1, 2, 4])
print(c.getNext())

class SingleElementCollection(Collection):
    def __init__(self, elements) -> None:
        super().__init__(elements)
        self.element = elements

    def getNext(self):
        return self.element

c = SingleElementCollection([1, 2, 4])
# for i in range(10):
#     print(c.getNext())

class LoopCollection(Collection):
    def __init__(self, elements) -> None:
        super().__init__(elements)
        self.elemnts = elements
        self.index = 0

    def getNext(self):
        element = self.elemnts[self.index]
        self.index = (self.index + 1) % len(self.elemnts)
        return element

c = LoopCollection([1, 2, 4])

def iterate(collection: Collection, n: int, f: function):
    for i in range(n):
        f(collection.getNext())

iterate(c, 10, print)

None
1
2
4
1
2
4
1
2
4
1


In [20]:
getmro(list)

(list, object)

In [21]:
class Collection(ABC):
    @abstractmethod
    def __init__(self, elements) -> None:
        pass

    @abstractmethod
    def getNext(self):
        pass

class SingleElement(Collection):
    def __init__(self, elements) -> None:
        self.element = elements

    def getNext(self):
        return self.element

class LoopCollection(Collection):
    def __init__(self, elements) -> None:
        self.elements = elements
        self.i = 0

    def getNext(self):
        element = self.elements[self.i]
        self.i %= len(self.elements)
        return element

c = LoopCollection([1, 2, 4])

def iterate(collection: Collection, n: int, f):
    for i in range(n):
        f(collection.getNext())

iterate(c, 10, print)

1
1
1
1
1
1
1
1
1
1


Wczytaj plik za pomocą `reader`, przetwórz treść za pomocą `converter` i zapisz do pliku za pomocą `writer`. Stwórz klasę `LUFileConverter`, która będzie konwertowała zadany plik zmieniając małe litery w wielkie i vice versa. Stwórz klasę `DoubleFileConverter`, która wykona operacje `converter` dwukrotnie.

`convertFile` w `FileConverter` ma być metodą abstrakcyjną.
`reader` ma posiadać operację `read()`, która zczyta obiekt typu `str` z pewnego pliku.
`writer` ma posiadać operację `write()`, która zapisze do pewnego pliku zadany obiekt typu `str`.
`converter` ma przyjąć obiekt typu `str` i zwrócić obiekt typu `str`.

 - Stwórz plik (`dummy.txt` lub jakikolwiek inny) z dowolną treścią (`Ala ma kota` lub cokolwiek innego).
 - Wczytaj treść tego pliku za pomocą operacji `open` i `read`.
 - Zapisz treść tego pliku za pomocą `open` i `write` do innego pliku (`temp.file` lub jakikolwiek inny).
 - Stwórz funkcję `toUpper`, która przyjmie obiekt typu `str` i zwróci ów tekst z wszystkimi małymi literami zamienionymi na wielkie.
 - Stwórz obiekt typu `FileConverter` i zapisz jako jego pola `reader`, `writer` oraz `toUpper`.
 - Zmień pole `convertFile` w `FileConverter` na metodę abstrakcyjną.

In [22]:
from abc import ABC, abstractmethod

def toUpper(text: str) -> str:
    return text.upper()

class FileConverter(ABC):
    def __init__(self, reader, writer, converter) -> None:
        self.reader = reader
        self.writer = writer
        self.converter = converter

    @abstractmethod
    def convertFile(self):
        pass

class UpperFileConverter(FileConverter):
    def __init__(self, reader, writer) -> None:
        super().__init__(reader, writer, toUpper)

    def convertFile(self):
        content = self.reader.read()
        content = self.converter(content)
        self.writer.write(content)

class DoubleFileConverter(FileConverter):
    def convertFile(self):
        content = self.reader.read()
        content = self.converter(content)
        content = self.converter(content)
        self.writer.write(content)

In [23]:
def doubleText(text):
    return text + text

in_file = "dummy.txt"
out_file = "temp.file"

with open(in_file, "w") as writer:
    writer.write("Ala ma kota\nA sierotka ma rysia\n\n")

with open(in_file, "r") as reader:
    with open(out_file, "w") as writer:
        fc = DoubleFileConverter(reader, writer, doubleText)
        fc.convertFile()

if os.path.exists(in_file):
    os.remove(in_file)
if os.path.exists(out_file):
    os.remove(out_file)

### Hermetyzacja

Niektóre pola nie powinny być dostępne z zewnątrz. Jeżeli nasza klasa wykonuje jakąś operację na własnych parametrach możemy chcieć chronić te parametry przed zmianą z zewnątrz lub przez klasy dziedziczące.

Pola [prywatne](https://en.wikipedia.org/wiki/Access_modifiers) są dostępne tylko dla klasy zawierającej owe pole. Klasy dziedziczące nie mają do tego pola dostępu. Podejście takie nazywamy hermetyzacją. Python nie posiada mechanizmu hermetyzacji i pola są zawsze dostępne. Jeżeli chcemy zaznaczyć, że pole jest zamierzone jako chronione zaczynamy jego nazwę od podkreślnika (`_`), jeżeli zaś chcemy, żeby było traktowane jako prywatne zaczynamy jego nazwę od podwójnego podkreślnika (`__`).

In [24]:
class ProtectedDummy:
    def __init__(self) -> None:
        self._counter = 0

    def add(self):
        self._counter += 1

    def getCounter(self):
        return self._counter

class ProtectedChild(ProtectedDummy):
    def add(self):
        self._counter += 3

dummy = ProtectedDummy()
dummy.add()
dummy.add()
dummy.add()
dummy._counter = -3
dummy.add()
dummy.add()
dummy.add()
dummy.getCounter()

0

In [25]:
class PrivateDummy:
    def __init__(self) -> None:
        self.__private = 0

    def add(self):
        self.__private += 1

    def counter(self):
        return self.__private

dummy = PrivateDummy()
dir(dummy)

dummy.add()
dummy.add()
dummy.add()
dummy._PrivateDummy__private = -3
dummy.add()
dummy.add()
dummy.add()
print(dummy.counter())

0


### Dataclass

In [26]:
from dataclasses import dataclass

@dataclass
class Dummy:
    a: int
    b: int
    c: int

dummy = Dummy(1, 2, 3)
dummy2 = Dummy(1, 2, 3)
print(dummy)
print(dummy == dummy2)
print(dummy.a, dummy.b, dummy.c)

Dummy(a=1, b=2, c=3)
True
1 2 3


### Kopia płytka a głęboka

## Serializacja

Serializacja do tekstu. Bardzo trudna do deserializacji.

In [27]:
data = [1, 2, 3]

dummy_file_name = "dummy.txt"

with open(dummy_file_name, "w") as writer:
    writer.write(str(data))

with open(dummy_file_name, "r") as reader:
    l = reader.read()

if os.path.exists(dummy_file_name):
    os.remove(dummy_file_name)

list(l)

['[', '1', ',', ' ', '2', ',', ' ', '3', ']']

Serializacja do postaci binarnej za pomocą modułu Pickle.

In [28]:
import pickle

data = [1, 2, 3]

dummy_file_name = "dummy.txt"

with open(dummy_file_name, "wb") as writer:
    pickle.dump(data, writer)

with open(dummy_file_name, "rb") as reader:
    l = pickle.load(reader)

if os.path.exists(dummy_file_name):
    os.remove(dummy_file_name)

l[1]

2

JSON

In [29]:
import json

dummy_file_name = "dummy.json"

data = {
    "capitals":
    {
        "Poland": "Warsaw",
        "Sri Lanka": "Sri Djayavardanapura Kotte",
        "Czechia": "Prague"
    },
    "numbers": [1, 2, 3, 4],
    "number": 15
}

with open(dummy_file_name, "w") as writer:
    json.dump(data, writer)

with open(dummy_file_name, "r") as reader:
    l = json.load(reader)

if os.path.exists(dummy_file_name):
    os.remove(dummy_file_name)

l["number"]


15

csv

In [30]:
dummy_file_name = "pracownicy.csv"

data = [['pracownik', 'pensja'],
    ['Jan Kowalski', '2500'],
    ['Joanna Dolegiewicz', '7500'],
    ['Katarzyna Kot', '10000']]

import csv

with open(dummy_file_name, 'w', newline='') as writer:
    csv_writer = csv.writer(writer)
    csv_writer.writerows(data)

with open(dummy_file_name, "r") as reader:
    csv_reader = csv.reader(reader)
    l = list(csv_reader)

if os.path.exists(dummy_file_name):
    os.remove(dummy_file_name)

l

[['pracownik', 'pensja'],
 ['Jan Kowalski', '2500'],
 ['Joanna Dolegiewicz', '7500'],
 ['Katarzyna Kot', '10000']]

## Dekoratory

## Wyrażenia lambda

## Generatory i iteratory

## Wyrażenia regularne

## Wielowątkowość

## Profilowanie kodu